*** PREPROCESAMIENTO ***

Datos del Drive. 
1. Conexión y permisos con el drive personal

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


2. Ubicación de las carpetas a utilizar

In [2]:
#%cd /content/gdrive/My Drive/0_Desarrollo_Tesis/2.Resultados_Preprocesamiento/
%cd /content/gdrive/My Drive/0_Desarrollo_Tesis/

/content/gdrive/.shortcut-targets-by-id/13aHaot6tbm7SROiS_JzQgfDtjZ_DM4PH/0_Desarrollo_Tesis


3. Listar la lista del contenido de la ruta

In [3]:
!ls /content/gdrive/My\ Drive/0_Desarrollo_Tesis/*

'/content/gdrive/My Drive/0_Desarrollo_Tesis/caracteristicas_globales_dataset.csv'
'/content/gdrive/My Drive/0_Desarrollo_Tesis/histogram_dataset.csv'

'/content/gdrive/My Drive/0_Desarrollo_Tesis/0.Detector Convolucional':
dogHeadDetector.dat

'/content/gdrive/My Drive/0_Desarrollo_Tesis/0. Estadisticas':
1.Datasets.csv			      3.Resultados_ext_caracteristicas.gsheet
1.Datasets.gsheet		      4.Dataset_control.csv
2.Resultados_Preprocesamiento.csv     4.Dataset_control.gsheet
2.Resultados_Preprocesamiento.gsheet  Estadísticas.pptx
3.Resultados_ext_caracteristicas.csv

'/content/gdrive/My Drive/0_Desarrollo_Tesis/10. Documentación':
 Cronogramas   Documento	  Guias			    Links.docx
 Dialogos     'Estado del arte'  'Investigación de Temas'

'/content/gdrive/My Drive/0_Desarrollo_Tesis/11.Presentaciones':
'Conceptos Web.pptx'   Invarianza.pptx	 Tesis

'/content/gdrive/My Drive/0_Desarrollo_Tesis/1.Programas_Funcionando':
 0.Estadisticas.ipynb
 1_Renombrar_imagenes.ipynb
'2_Preprocesami

Resultados del preprocesamiento son las siguientes carpetas:


1.   NO_DETECTADAS: En esta carpeta se ubicaran las imagenes la cuales no se detecta la parte facial del perro/gato.
2.   DETECTADAS: En esta carpeta se ubicaran las imagenes que fueron detectadas y donde se señala la parte facial del perro/gato.
3.   ROSTRO_REDIMENSIONADO: En esta carpeta se ubicaran las imagenes que contienen unicamente la región del rostro del perro/gato. Las guarda en una dimension (180 x 180) y formato .png

In [4]:
# Importar Librerias

import dlib, cv2, os
from imutils import face_utils
import numpy as np
import matplotlib.pyplot as plt
import os

In [5]:
def menu_tesis():
   print("----------- Tesis U Catolica -----------")
   print("1. Perros")
   print("2. Buscar Perro")
   print("3. Gatos")
   print("4. Buscar Gato")
   print("-----------------------------------")

In [6]:
def seleccion_opcion():
  menu_tesis()
  opcion = int(input("Seleccione una opcion: "))
  caso_opcion(opcion)

In [7]:

def preprocesamiento_imagen(path_DATASET,path_DETECTADAS,path_NO_DETECTADAS,path_ROSTRO_REDIMENSIONADO,path_ANTES_RNC):
  cont_imagen=0

  #CREACION DE ARCHIVOS CSV 
  Ancho_Alto_RNC="2.Resultados_Preprocesamiento/Ancho_Alto_RNC.csv"

  #Guarda la ruta de imagen REDIMENSIONADA
  archivo_ancho_alto_RNC= open(Ancho_Alto_RNC, "w+") 

  archivo_ancho_alto_RNC.write("IMAGEN,ANCHO,ALTO \n")

  #For que recorre las carpetas(sujetos) del path_DATASET
  for cj in os.listdir(path_DATASET):
      
      #For que recorre la imagen de cada carpeta(sujeto)
      for nj in os.listdir(path_DATASET + '/' + cj):

        cont_imagen=cont_imagen+1
        #-------------
        #cj   -> Carpeta del sujeto 
        #nj   -> Nombre de la imagen del sujeto
        #path_DATASET -> Ruta de la imagen
        imgPath = path_DATASET + '/' + cj + '/' + nj
        #-------------
      
        #Abrir Imagen (Lee la imagen en formtado BGR)
        imgBGR = cv2.imread(imgPath)
        #print(imgPath)
        #print(imgBGR)
        print(cj)
        #print(nj)
        height, width, channels = imgBGR.shape
        #print("|---Imagen de entrada----|")
        #print ("Alto: ",height," Ancho: ", width,"Canales: ", channels)
        #print ("Resultado", height*width)
        total=(height*width)

        #Cambiar el tamaño resize
        if(total>=1310720):
          #-----------------------------------------------
          #Guardar imagenes que aplican este filtro. 
          #----------------------------------------------
          os.makedirs(path_ANTES_RNC+ '/'+cj, exist_ok=True)
          filename=(path_ANTES_RNC+ '/' + cj + '/' + nj) 
          cv2.imwrite(filename, imgBGR) 
          #-----------------------------------------------
          #resize. 
          #----------------------------------------------
          height = 700
          width = 700
          dim = (width, height)
          imgBGR = cv2.resize(imgBGR, dim, interpolation=cv2.INTER_LINEAR)
          

        #Convierte la imagen en RGB
        imgRGB = cv2.cvtColor(imgBGR, cv2.COLOR_BGR2RGB)

        #Copiar imagen del formato RGB para luego convertirla en GRAY
        imgcopyBGR_cortar = np.copy(imgBGR)
        imgcopyBGR_cuadros = np.copy(imgBGR)

        #Nombre y extension
        filename, ext = os.path.splitext(os.path.basename(imgPath))

        #Copiar imagen del formato RGB para luego convertirla en GRAY
        imgcopyRGB = np.copy(imgRGB)
        # Se convierte a escala de grises 
        imgGRAY = cv2.cvtColor(imgcopyRGB, cv2.COLOR_RGB2GRAY)

        
        #--------------------------------------------------------------
        #Detector de Cara
        #https://github.com/kairess/dog_face_detector
        detector = dlib.cnn_face_detection_model_v1('0.Detector Convolucional/dogHeadDetector.dat')
        dets = detector(imgGRAY, upsample_num_times=1)


        #print("Cantidad de caras encontradas en la Imagen->",len(dets))

        if (len(dets)==0):
            #print ("Error: No se detectaron caras en la imagen: ", imgi)
            #Guarda imagenes imagenes NO ENCONTRADAS
            os.makedirs(path_NO_DETECTADAS+ '/'+ cj, exist_ok=True)
            filename=(path_NO_DETECTADAS+ '/' + cj + '/' + nj ) 
            #print(filename)
            cv2.imwrite(filename, imgBGR) 
        else:

          #Recorre el número de caras encontradas
          for i, d in enumerate(dets):

              #print("Detection {}: Left: {} Top: {} Right: {} Bottom: {} Confidence: {}".format(i, d.rect.left(), d.rect.top(), d.rect.right(), d.rect.bottom(), d.confidence))

              x, y = d.rect.left(), d.rect.top()
              x2, y2 = d.rect.right(), d.rect.bottom()
              #alto
              h=x2-x
              #ancho
              w=y2-y
              # representa la esquina superior izquierda del rectángulo PT1
              # representa la esquina inferior izquierda del rectángulo PT2
              # Color (255,0,0) en RGB
              # Grosor de línea thickness
              # Se usa el método cv2.rectangle () para dibujar un rectángulo
              cv2.rectangle(imgcopyBGR_cuadros, pt1=(x, y), pt2=(x2, y2), thickness=2, color=(255, 0, 0), lineType=cv2.LINE_AA)

              archivo_ancho_alto_RNC.write(path_ROSTRO_REDIMENSIONADO+ '/' + str(cj) + '/' + str(nj))
              archivo_ancho_alto_RNC.write(",")
              archivo_ancho_alto_RNC.write(str(w))
              archivo_ancho_alto_RNC.write(",")
              archivo_ancho_alto_RNC.write(str(h))
              archivo_ancho_alto_RNC.write("\n")


              #Se valida si las coordenadas son negativas para dejar en cero por defecto
              if y<0:
                  y=0
              if x<0:
                  x=0
              if h<0:
                  h=0
              if w<0:
                  w=0

              #Corta y guarda imagenes
              # Crear la carpeta si no esta creada 
              os.makedirs(path_ROSTRO_REDIMENSIONADO+ '/'+cj, exist_ok=True)
              filename=(path_ROSTRO_REDIMENSIONADO+ '/' + cj + '/' + nj) 
              #Ejemplo: image[y:y+h, x:x+w]
              cropped_image=imgcopyBGR_cortar[y:(y+h), x:(x+w)]
              #Cambiar el tamaño resize
              height = 180
              width = 180
              dim = (width, height)
              cropped_image = cv2.resize(cropped_image, dim, interpolation=cv2.INTER_LINEAR)
              cv2.imwrite(filename, cropped_image) 
          #--------------------------------------------------------------

          
          #Guarda imagenes imagenes con cuadros
          os.makedirs(path_DETECTADAS+ '/'+cj, exist_ok=True)
          filename=(path_DETECTADAS+ '/' + cj + '/' + nj) 
          cv2.imwrite(filename, imgcopyBGR_cuadros) 

  print("TERMINO !!")
  print("Cantidad de imagenes ", cont_imagen)

  #plt.figure(figsize=(16, 16))
  #plt.imshow(imgcopyBGR_cuadros)

In [8]:
def caso_opcion(opcion):
  if opcion == 1:
    print('Perros')
    path_DATASET = '3.Datasets/1.DATASET_PERROS' #1
    path_DETECTADAS='2.Resultados_Preprocesamiento/Perros/DETECTADAS' #2
    path_NO_DETECTADAS='2.Resultados_Preprocesamiento/Perros/NO_DETECTADAS' #3
    path_ROSTRO_REDIMENSIONADO='2.Resultados_Preprocesamiento/Perros/ROSTRO_REDIMENSIONADO' #4
    path_ANTES_RNC='2.Resultados_Preprocesamiento/Perros/ANTES_RNC' #5
    preprocesamiento_imagen(path_DATASET,path_DETECTADAS,path_NO_DETECTADAS,path_ROSTRO_REDIMENSIONADO,path_ANTES_RNC)
  elif opcion == 2:
    print('Buscar Perro')
    path_DATASET = '3.Datasets/3.DATASET_ENTRADA_PERROS' #1
    path_DETECTADAS='2.Resultados_Preprocesamiento/Buscar_perro/DETECTADAS' #2
    path_NO_DETECTADAS='2.Resultados_Preprocesamiento/Buscar_perro/NO_DETECTADAS' #3
    path_ROSTRO_REDIMENSIONADO='2.Resultados_Preprocesamiento/Buscar_perro/ROSTRO_REDIMENSIONADO' #4
    path_ANTES_RNC='2.Resultados_Preprocesamiento/Buscar_perro/ANTES_RNC' #5
    preprocesamiento_imagen(path_DATASET,path_DETECTADAS,path_NO_DETECTADAS,path_ROSTRO_REDIMENSIONADO,path_ANTES_RNC)
  elif opcion == 3:
    print('Gatos')
    path_DATASET = '3.Datasets/2.DATASET_GATOS' #1
    path_DETECTADAS='2.Resultados_Preprocesamiento/Gatos/DETECTADAS' #2
    path_NO_DETECTADAS='2.Resultados_Preprocesamiento/Gatos/NO_DETECTADAS' #3
    path_ROSTRO_REDIMENSIONADO='2.Resultados_Preprocesamiento/Gatos/ROSTRO_REDIMENSIONADO' #4
    path_ANTES_RNC='2.Resultados_Preprocesamiento/Gatos/ANTES_RNC' #5
    preprocesamiento_imagen(path_DATASET,path_DETECTADAS,path_NO_DETECTADAS,path_ROSTRO_REDIMENSIONADO,path_ANTES_RNC)
                            
  elif opcion == 4:
    print('Buscar Gato')
    path_DATASET = '3.Datasets/4.DATASET_ENTRADA_GATOS' #1
    path_DETECTADAS='2.Resultados_Preprocesamiento/Buscar_gato/DETECTADAS' #2
    path_NO_DETECTADAS='2.Resultados_Preprocesamiento/Buscar_gato/NO_DETECTADAS' #3
    path_ROSTRO_REDIMENSIONADO='2.Resultados_Preprocesamiento/Buscar_gato/ROSTRO_REDIMENSIONADO' #4
    path_ANTES_RNC='2.Resultados_Preprocesamiento/Buscar_gato/ANTES_RNC' #5
    preprocesamiento_imagen(path_DATASET,path_DETECTADAS,path_NO_DETECTADAS,path_ROSTRO_REDIMENSIONADO,path_ANTES_RNC)
  else:
    #seleccion_opcion()
    print('xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx')
    print('Error: La opción ingresada no esta dentro de las opciones, por favor vuelva a ingresar la opción')
    print('xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx')
    print('')
    print('')
    print('')
    seleccion_opcion()

seleccion_opcion()

----------- Tesis U Catolica -----------
1. Perros
2. Buscar Perro
3. Gatos
4. Buscar Gato
-----------------------------------
Seleccione una opcion: 3
Gatos
1_Cat_Persa
1_Cat_Persa
1_Cat_Persa
1_Cat_Persa
1_Cat_Persa
2_Celestina_Criollo
2_Celestina_Criollo
2_Celestina_Criollo
2_Celestina_Criollo
3_Lola_Criollo
3_Lola_Criollo
3_Lola_Criollo
3_Lola_Criollo
3_Lola_Criollo
3_Lola_Criollo
3_Lola_Criollo
3_Lola_Criollo
3_Lola_Criollo
3_Lola_Criollo
3_Lola_Criollo
3_Lola_Criollo
3_Lola_Criollo
3_Lola_Criollo
3_Lola_Criollo
3_Lola_Criollo
3_Lola_Criollo
3_Lola_Criollo
3_Lola_Criollo
4_Kira_Criollo
4_Kira_Criollo
5_Kuroneko_Criollo
5_Kuroneko_Criollo
5_Kuroneko_Criollo
5_Kuroneko_Criollo
5_Kuroneko_Criollo
6_Lupita_Criollo
6_Lupita_Criollo
6_Lupita_Criollo
6_Lupita_Criollo
6_Lupita_Criollo
6_Lupita_Criollo
6_Lupita_Criollo
6_Lupita_Criollo
6_Lupita_Criollo
6_Lupita_Criollo
7_Luna_Criollo
7_Luna_Criollo
7_Luna_Criollo
7_Luna_Criollo
8_Ramon_Criollo
8_Ramon_Criollo
8_Ramon_Criollo
8_Ramon_Crioll